## 내장된 메서드로 훈련 및 평가하기

훈련 및 유효성 검증을 위해 내장 API를 사용할 때의 훈련, 평가 및 예측(추론) 모델에 대한 설명

In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## API 개요 : 첫 번째 엔드 투 엔드 예제

* 데이터를 모델의 내장 훈련 루프로 전달할 때는 NumPy 배열 또는 `tf.data Dataset` 객체를 사용해야 함

In [31]:
inputs = keras.Input(shape=(784,), name="digits")

# Dense Layer: 입력 레이어 1개, 은닉 레이어 2개, 출력 레이더 1개로 구성 (완전 연결층이라고 정의)
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

# 모델 생성
model = keras.Model(inputs=inputs, outputs=outputs)

**일반적인 엔드 투 엔드 워크플로**

1. 훈련
2. 원래 훈련 데이터에서 생성된 홀드아웃 세트에 대한 유효성 검사
3. 테스트 데이터에 대한 평가

예) 다음 예시는 MNIST 데이터세트를 NumPy 배열로 사용

In [32]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# 데이터를 NumPy 배열로 변환
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Validation을 위한 10000개의 샘플 보류
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

훈련 구성 (최적화, 손실, 메트릭) 지정

In [33]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # 최적화 (Optimizer)
    # 손실함수 (SparseCategoricalCrossentropy)
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # 메트릭 (List of metrics to monitor)
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

`fit()`을 호출하여 데이터를 배치로 분할하고, `epochs`에 대해 전체 데이터세트를 반복처리하여 모델 훈련

In [34]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # 각 에포크가 끝날 때 손실, 메트릭을 모니터링하기 위해 일부 validation 은 통과
    validation_data=(x_val, y_val),
)

Fit model on training data
Epoch 1/2
782/782 [==============================] - 2s 3ms/step - loss: 0.3374 - sparse_categorical_accuracy: 0.9039 - val_loss: 0.1962 - val_sparse_categorical_accuracy: 0.9437
Epoch 2/2
782/782 [==============================] - 2s 2ms/step - loss: 0.1579 - sparse_categorical_accuracy: 0.9532 - val_loss: 0.1482 - val_sparse_categorical_accuracy: 0.9572


반환되는 `history` 객체는 훈련 중 손실 값과 메트릭 값에 대한 레코드 유지

In [35]:
history.history

{'loss': [0.337371289730072, 0.1578771024942398],
 'sparse_categorical_accuracy': [0.9039400219917297, 0.9531999826431274],
 'val_loss': [0.19617846608161926, 0.14819879829883575],
 'val_sparse_categorical_accuracy': [0.9437000155448914, 0.9571999907493591]}

`evaluate()`를 통해 테스트 데이터에 대해 모델 평가

In [36]:
# `evaluate`을 사용하여 테스트 데이터에 대한 모델 평가
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# 새로운 데이터를 사용하여 예측
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
79/79 [==============================] - 0s 2ms/step - loss: 0.1480 - sparse_categorical_accuracy: 0.9545
test loss, test acc: [0.14796650409698486, 0.9545000195503235]
Generate predictions for 3 samples
1/1 [==============================] - 0s 73ms/step
predictions shape: (3, 10)


## compile() 메서드 : 손실, 메트릭 및 최적화 프로그램 지정하기

`fit()`으로 모델 훈련을 시키기 위해 손실 함수, 최적화 프로그램, 그리고 선택적으로 모니터링할 일부 메트릭을 지정해야 함

이러한 내용들은 `compile()` 메서드에 대한 인수로 모델에 전달해야 함

In [37]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

`metrics` 인수는 목록이어야 함 (모델 메트릭 수는 얼마든지 가능)


모델에 여러 개의 출력이 있는 경우, 각 출력에 대해 로 다른 손실 및 메트릭을 지정하고 모델의 총 손실에 대한 각 출력의 기여도를 조정할 수 있음

In [38]:
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

나중에 재사용하기 위해 모델 정의와 컴파일 단계를 함수에 넣음

In [39]:
# 컴파일하지 않은 모델
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# 컴파일 한 모델
def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    return model

### 많은 내장 최적화 프로그램, 손실 및 메트릭 사용 가능

* 최적화 프로그램 : `SGD()`, `RMSprop()`, `Adam()`, ...


* 손실 : `MeanSquaredError()`, `KLDivergence()`, `CosineSimilarity()`, ...


* 메트릭 : `AUC()`, `Precision()`, `Recall()`, ...

### 사용자 정의 손실

사용자 정의 손실을 만들어야 하는 경우, Keras는 두 가지 방법 제공


1. 입력 y_true 및 y_pred를 받아들이는 함수를 만듦

In [40]:
# 실제 데이터와 예측 사이의 오차에 대해 평균 제곱을 계산하는 손실함수
def custom_mean_squared_error(y_true, y_pred):
    return tf.math.reduce_mean(tf.square(y_true - y_pred))

model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=custom_mean_squared_error)

# MSE를 사용하기 위해 원-핫 인코딩
y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)

782/782 [==============================] - 2s 2ms/step - loss: 0.0160


2. y_true 및 y_pred 이외의 매개변수를 사용하는 손실 함수가 필요한 경우 `tf.keras.losses.Loss` 클래스를 하위 클래스화하고 다음 두 메서드를 구현


- `__init__(self)` : 손실 함수 호출 중에 전달할 매개변수를 받아들임


- `call(self, y_true, y_pred)` : 목표(y_true)와 모델 예측(y_pred)을 사용하여 모델의 손실을 계산

오차의 평균 제곱을 사용하려고 하지만 예측 값을 0.5에서 멀어지게 하는 항이 추가되었다고 가정 (범주형 목표가 원-핫 인코딩되고 0과 1 사이의 값을 취하는 것으로 가정) 


이렇게 하면 모델이 너무 확신을 갖지 않게 하는 인센티브가 *과적합*을 줄이는 데 도움이 될 수 있음 (단, 시도 하기 전까진 효과가 있는지 알 수 없음)

In [41]:
class CustomMSE(keras.losses.Loss):
    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super().__init__(name=name)
        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        reg = tf.math.reduce_mean(tf.square(0.5 - y_pred))  # 예측 값을 0.5에서 멀어지게 함
        return mse + reg * self.regularization_factor


model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=CustomMSE())

y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)

782/782 [==============================] - 2s 2ms/step - loss: 0.0385


### 사용자 정의 메트릭

`tf.keras.metrics.Metric` : API의 일부가 아닌 메트릭이 필요한 경우, 이 클래스를 하위 클래스화하여 사용자 정의 메트릭 생성 가능


* *아래의 4가지 메서드 구현 필요*

    1. `__init__(self)` : 메트릭에 대한 상태 변수 생성
    2. `update_state(self, y_true, y_pred, sample_weight=None)` : y_true 목표값과 y_pred 모델 예측값을 사용하여 상태 변수 업데이트
    3. `result(self)` : 상태 변수를 사용하여 최종 경과 계산
    4. `reset_state(self)` : 메트릭 상태를 다시 초기화 함


* 상태 업데이트와 결과 계산은 개별적으로 수행 -> 일부의 경우, 결과 계산 부담이 크고, 주기적으로 수행되기 때문

In [42]:
# 올바르게 분류된 샘플 수를 계산하는 메트릭 구현 예제
class CategoricalTruePositives(keras.metrics.Metric):
    # 메트릭에 대한 상태 변수 생성
    def __init__(self, name="categorical_true_positives", **kwargs):
        super(CategoricalTruePositives, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="ctp", initializer="zeros")

    # 상태 변수 업데이트 (y_true, y_pred 사용)
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
        values = tf.cast(y_true, "int32") == tf.cast(y_pred, "int32")
        values = tf.cast(values, "float32")
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))
    
    # 상태 변수를 사용하여 최종 경과 계산
    def result(self):
        return self.true_positives

    # 메트릭 상태를 다시 초기화
    def reset_state(self):
        # epoch 시작마다 메트릭 상태 초기화
        self.true_positives.assign(0.0)

model = get_uncompiled_model()  # 컴파일 되지 않은 모델 함수
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[CategoricalTruePositives()],
)
model.fit(x_train, y_train, batch_size=64, epochs=3)

Epoch 1/3
782/782 [==============================] - 2s 2ms/step - loss: 0.3461 - categorical_true_positives: 45041.0000
Epoch 2/3
782/782 [==============================] - 1s 2ms/step - loss: 0.1591 - categorical_true_positives: 47641.0000
Epoch 3/3
782/782 [==============================] - 2s 2ms/step - loss: 0.1170 - categorical_true_positives: 48234.0000


### 표준 서명에 맞지 않는 손실 및 메트릭 처리

* 거의 대부분의 손실과 메트릭은 `y_true`와 `y_pred`에서 계산할 수 있지만 모두 그런 것은 아님


* 이러한 경우, 사용자 정의 레이어의 호출 메서드 내에서 `self.add_loss(loss_value)`를 호출 가능<br> 추가된 손실은 훈련 중 "주요" 손실(compile()로 전달되는 손실)에 추가

In [43]:
# 활동 정규화를 추가하는 간단한 예
class ActivityRegularizationLayer(layers.Layer):
    def call(self, inputs):
        self.add_loss(tf.reduce_sum(inputs) * 0.1)
        return inputs  # Pass-through layer.


inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)

# Layer에 활동 정규화 삽입
x = ActivityRegularizationLayer()(x)

x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

# 정규화 요소를 추가했으므로, 보여지는 손실은 전보다 더 커짐
model.fit(x_train, y_train, batch_size=64, epochs=1)

782/782 [==============================] - 2s 2ms/step - loss: 2.5208


* `add_metric()`을 사용하여 메트릭 값 로깅에 대해 같은 작업 수행 가능

In [44]:
class MetricLoggingLayer(layers.Layer):
    def call(self, inputs):
        # `aggregation`은 각 epoch마다 각 배치 값을 합치는 방법을 정의
        self.add_metric(
            keras.backend.std(inputs), name="std_of_activation", aggregation="mean"
        )
        return inputs  # Pass-through layer.

inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)

# Insert std logging as a layer.
x = MetricLoggingLayer()(x)

x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)
model.fit(x_train, y_train, batch_size=64, epochs=1)

782/782 [==============================] - 2s 2ms/step - loss: 0.3516 - std_of_activation: 0.9914


* Functional API에서 `model.add_loss(loss_tensor)` 또는 `model.add_metric(metric_tensor, name, aggregation)`을 호출 가능
    - `add_loss()`를 통해, 손실 전달하면 모델에 이미 최소화할 수 있는 손실이 있으므로 손실 함수 없이 `compile()` 호출 가능

In [45]:
inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x2 = layers.Dense(64, activation="relu", name="dense_2")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

# Function API
model.add_loss(tf.reduce_sum(x1) * 0.1)

model.add_metric(keras.backend.std(x1), name="std_of_activation", aggregation="mean")

model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)
model.fit(x_train, y_train, batch_size=64, epochs=1)

782/782 [==============================] - 2s 2ms/step - loss: 2.5171 - std_of_activation: 0.0020


`LogisticEndpoint` 레이어 : 입력으로  targets 및 logits를 받아들이고 `add_loss()`를 통해 교차 엔트로피 손실을 추적

- `add_metric()`을 통해 분류 정확도도 추적

In [46]:
class LogisticEndpoint(keras.layers.Layer):
    def __init__(self, name=None):
        super(LogisticEndpoint, self).__init__(name=name)
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
        self.accuracy_fn = keras.metrics.BinaryAccuracy()

    def call(self, targets, logits, sample_weights=None):
        # training-time loss 값을 계산하고
        # 'self.add_loss()'를 사용한 레이어를 추가
        loss = self.loss_fn(targets, logits, sample_weights)
        self.add_loss(loss)
        
        # 메트릭으로 정확도를 기록하고
        # 'self.add_metric()'을 사용하여 레이어에 추가
        acc = self.accuracy_fn(targets, logits, sample_weights)
        self.add_metric(acc, name="accuracy")

        # 추론 시간 예측 텐서 반환 (.predict()에 대해)
        return tf.nn.softmax(logits)

* loss 인수 없이 컴파일된 두 개의 입력(입력 data 및 targets)이 있는 모델에서 사용 가능

In [47]:
import numpy as np

inputs = keras.Input(shape=(3,), name="inputs")
targets = keras.Input(shape=(10,), name="targets")
logits = keras.layers.Dense(10)(inputs)
predictions = LogisticEndpoint(name="predictions")(logits, targets)

model = keras.Model(inputs=[inputs, targets], outputs=predictions)
model.compile(optimizer="adam")  # No loss argument!

data = {
    "inputs": np.random.random((3, 3)),
    "targets": np.random.random((3, 10)),
}
model.fit(data)

1/1 [==============================] - 0s 371ms/step - loss: 0.9727 - binary_accuracy: 0.0000e+00


### 유효성 검사 홀드아웃 세트를 자동으로 분리

1. `validation_data` 인수를 사용하여 NumPy 배열의 튜플 `(x_val, y_val)`을 모델에 전달하여 각 epoch 끝에서 유효성 검사 손실과 유효성 검사 메트릭을 평가


2. 인수 `validation_split`를 사용하여 검증 목적으로 훈련 데이터의 일부를 자동으로 예약 가능.<br> *인수 값은 검증을 위해 예약할 데이터 비율을 나타내므로, 0보다 크고 1보다 작은 값으로 설정해야 함 <br> (예: validation_split=0.2는 "유효성 검사를 위해 데이터의 20%를 사용"한다는 의미)* <br> => NumPy 데이터로 훈련할 때만 사용 가능 !!


- 유효성 계산 방법 : 셔플링 전에 `fit()` 호출로 수신한 배열의 마지막 x% 샘플을 가져오는 것

In [48]:
model = get_compiled_model()
model.fit(x_train, y_train, batch_size=64, validation_split=0.2, epochs=1)

625/625 [==============================] - 2s 3ms/step - loss: 0.3726 - sparse_categorical_accuracy: 0.8947 - val_loss: 0.2353 - val_sparse_categorical_accuracy: 0.9277


## tf.data 데이터세트로부터 훈련 및 평가

데이터가 `tf.data.Dataset` 객체의 형태로 제공되는 경우

- `tf.data` API는 빠르고 확장 가능한 방식으로, 데이터를 로드하고 사전 처리하기 위한 세트
- `Dataset` 인스턴스를 메서드 `fit()`, `evaluate()`, `predict()`로 직접 전달 가능

In [49]:
model = get_compiled_model()

# 먼저, training Dataset 인스턴스 만듦 (MNIST 데이터를 사용할 것)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle and slice the dataset.
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Now we get a test dataset.
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(64)

# 데이터셋이 이미 배치되었으므로, 우리는 배치사이즈만큼 지나갈 수 없음
model.fit(train_dataset, epochs=3)

# You can also evaluate or predict on a dataset.
print("Evaluate")
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

Epoch 1/3
782/782 [==============================] - 2s 2ms/step - loss: 0.3353 - sparse_categorical_accuracy: 0.9049
Epoch 2/3
782/782 [==============================] - 1s 2ms/step - loss: 0.1543 - sparse_categorical_accuracy: 0.9543
Epoch 3/3
782/782 [==============================] - 1s 2ms/step - loss: 0.1128 - sparse_categorical_accuracy: 0.9665
Evaluate
157/157 [==============================] - 0s 2ms/step - loss: 0.1176 - sparse_categorical_accuracy: 0.9636


{'loss': 0.11762889474630356,
 'sparse_categorical_accuracy': 0.9635999798774719}

데이터 세트는 각 epoch의 끝에서 재설정되므로, 다음 epoch에서 재사용 가능

- 특정 배치 수에 대해서만 훈련을 실행하려면 다음 epoch로 이동하기 전에 이 데이터세트를 사용하여 모델이 실행해야 하는 훈련 단계의 수를 지정하는 `steps_per_epoch` 인수를 전달할 수 있음


- But, 이렇게 하면 각 epoch가 끝날 때 데이터세트가 재설정되지 않고 다음 배치를 계속 가져오게 됨. 무한 반복되는 데이터세트가 아니라면 결국 데이터세트의 데이터가 고갈.

In [50]:
model = get_compiled_model()

# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Only use the 100 batches per epoch (that's 64 * 100 samples)
model.fit(train_dataset, epochs=3, steps_per_epoch=100)

Epoch 1/3
100/100 [==============================] - 1s 2ms/step - loss: 0.7859 - sparse_categorical_accuracy: 0.7911
Epoch 2/3
100/100 [==============================] - 0s 2ms/step - loss: 0.3846 - sparse_categorical_accuracy: 0.8878
Epoch 3/3
100/100 [==============================] - 0s 2ms/step - loss: 0.3406 - sparse_categorical_accuracy: 0.8969


### 유효성 검사 데이터세트 사용

`fit()`에서 `Dataset` 인스턴스를 `validation_data` 인수로 전달

In [51]:
model = get_compiled_model()

# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Prepare the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

model.fit(train_dataset, epochs=1, validation_data=val_dataset)

782/782 [==============================] - 3s 3ms/step - loss: 0.3486 - sparse_categorical_accuracy: 0.9010 - val_loss: 0.2440 - val_sparse_categorical_accuracy: 0.9263


각 epoch가 끝날 때, 모델은 검증 데이터세트를 반복하고 유효성 검사 손실 및 검증 메트릭을 계산함

특정 배치 수에 대해서만 유효성 검사를 실행하려면 유효성 검사를 중단하고 다음 epoch로 넘어가기 전에 유효성 검사 데이터세트에서 모델이 실행해야 하는 유효성 검사 단계의 수를 지정하는 `validation_steps` 인수를 전달할 수 있음

In [52]:
model = get_compiled_model()

# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Prepare the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

model.fit(
    train_dataset,
    epochs=1,
    # Only run validation using the first 10 batches of the dataset
    # using the `validation_steps` argument
    validation_data=val_dataset,
    validation_steps=10,
)

782/782 [==============================] - 2s 2ms/step - loss: 0.3416 - sparse_categorical_accuracy: 0.9030 - val_loss: 0.3471 - val_sparse_categorical_accuracy: 0.9031


유효성 검사 데이터세트 : 사용 후 매번 재설정되므로 epoch마다 항상 동일한 샘플을 평가하게 됨

- 인수 `validation_split`(훈련 데이터로부터 홀드아웃 세트 생성)는 `Dataset` 객체로 훈련할 때는 지원되지 않는데, 이를 위해서는 데이터세트 샘플을 인덱싱할 수 있어야 하지만 `Dataset` API에서는 일반적으로 이것이 불가능하기 때문

## 지원되는 다른 입력 형식

**Keras 모델을 사용하는 경우**


- 데이터가 작고 메모리에 맞는 경우 `NumPy 입력 데이터`


- 큰 데이터세트가 있고 분산 훈련을 수행해야 하는 경우 `Dataset 객체`


- 큰 데이터세트가 있고 TensorFlow에서 수행할 수 없는 많은 사용자 정의 Python 측 처리를 수행해야 하는 경우(예: 데이터 로드 또는 사전 처리를 위해 외부 라이브러리에 의존하는 경우) `Sequence 객체`

## keras.utils.Sequence 객체를 입력으로 사용하기

`keras.utils.Sequence`

- 멀티 프로세싱과 잘 작동함
- 셔플 가능 (예: `fit()`에서 `shuffle=True`를 전달하는 경우)

Sequence는 두 가지 메서드 구현 필요

- `__getitem__` : 완전한 배치 반환 필요, epoch 사이에서 데이터 세트를 수정하려면 `on_epoch_end` 구현 가능
- `__len__`

In [53]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np

# filnames : 이미지 경로의 리스트
# labels : 연관된 라벨들

class CIFAR10Sequence(Sequence):
    def __init__(self, filenames, labels, batch_size):
        self.filenames, self.labels = filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        return np.array([
            resize(imread(filename), (200, 200))
               for filename in batch_x]), np.array(batch_y)

sequence = CIFAR10Sequence(filenames, labels, batch_size)
model.fit(sequence, epochs=10)

## 오류 발생 : Sequence 미선언 => CIFAR10Sequence 안에 있는 요소가 없음

NameError: name 'Sequence' is not defined

---
---

## 샘플 가중치 및 클래스 가중치 사용

데이터에 가중치를 부여하는 방법 : 클래스 가중치, 샘플 가중치

### 클래스 가중치

`Model.fit()`에 대한 `class_weight` 인수로 사전을 전달하여 설정

- 이 사전은 클래스 인덱스를 이 클래스에 속한 샘플에 사용해야 하는 가중치에 매핑
- 샘플링을 다시 수행하지 않고, 클래스의 균형을 맞추거나 특정 클래스에 더 중요한 모델을 훈련시키는 데 사용 가능

In [54]:
import numpy as np

class_weight = {
    0: 1.0,
    1: 1.0,
    2: 1.0,
    3: 1.0,
    4: 1.0,
    # Set weight "2" for class "5",
    # making this class 2x more important
    5: 2.0,
    6: 1.0,
    7: 1.0,
    8: 1.0,
    9: 1.0,
}

print("Fit with class weight")
model = get_compiled_model()
model.fit(x_train, y_train, class_weight=class_weight, batch_size=64, epochs=1)

Fit with class weight
782/782 [==============================] - 2s 2ms/step - loss: 0.3710 - sparse_categorical_accuracy: 0.9014


### 샘플 가중치

세밀한 제어가 필요하거나 분류기를 빌드하지 않는 경우 사용 가능

- NumPy 데이터에서 훈련하는 경우: `sample_weight` 인수를 `Model.fit()`으로 전달
- `tf.data` 또는 다른 종류의 반복기에서 훈련하는 경우: `Yield (input_batch, label_batch, sample_weight_batch)` 튜플을 생성

"샘플 가중치" 배열은 총 손실을 계산할 때 배치의 각 샘플에 필요한 가중치를 지정하는 숫자 배열로, 불균형적인 분류 문제에 일반적으로 사용 (거의 드러나지 않는 클래스에 더 많은 가중치를 부여한다는 개념)

사용된 가중치가 1과 0인 경우, 배열은 손실 함수에 대한 마스크로 사용될 수 있음 (전체 손실에 대한 특정 샘플의 기여도를 완전히 버림)

In [55]:
sample_weight = np.ones(shape=(len(y_train),))
sample_weight[y_train == 5] = 2.0

print("Fit with sample weight")
model = get_compiled_model()
model.fit(x_train, y_train, sample_weight=sample_weight, batch_size=64, epochs=1)

Fit with sample weight
782/782 [==============================] - 2s 2ms/step - loss: 0.3808 - sparse_categorical_accuracy: 0.9001


In [56]:
sample_weight = np.ones(shape=(len(y_train),))
sample_weight[y_train == 5] = 2.0

# Create a Dataset that includes sample weights
# (3rd element in the return tuple).
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train, sample_weight))

# Shuffle and slice the dataset.
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

model = get_compiled_model()
model.fit(train_dataset, epochs=1)

782/782 [==============================] - 2s 2ms/step - loss: 0.3886 - sparse_categorical_accuracy: 0.9000


## 다중 입력, 다중 출력 모델로 데이터 전달하기

입력 또는 출력이 여러 개인 모델

- 형상 `(32, 32, 3)`(height, width, channels)의 이미지 입력과 형상 `(None, 10)`(timesteps, features)의 시계열 입력이 있는 다음 모델을 고려해보면
- 이 모델은 이러한 입력의 조합으로부터 계산된 다음의 두 출력을 가짐. "score"(형상 `(1,)`) 및 다섯 개의 클래스에 걸친 확률 분포(형상 `(5,)`)

In [57]:
image_input = keras.Input(shape=(32, 32, 3), name="img_input")
timeseries_input = keras.Input(shape=(None, 10), name="ts_input")

x1 = layers.Conv2D(3, 3)(image_input)
x1 = layers.GlobalMaxPooling2D()(x1)

x2 = layers.Conv1D(3, 3)(timeseries_input)
x2 = layers.GlobalMaxPooling1D()(x2)

x = layers.concatenate([x1, x2])

score_output = layers.Dense(1, name="score_output")(x)
class_output = layers.Dense(5, name="class_output")(x)

model = keras.Model(
    inputs=[image_input, timeseries_input], outputs=[score_output, class_output]
)

이 모델을 플롯해 보면 여기서 수행 중인 작업을 명확하게 확인할 수 있음 (플롯에 표시된 형상이 샘플별 형상이 아니라 배치 형상임에 주목).

In [58]:
keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


컴파일할 때 손실 함수를 목록으로 전달하여 출력마다 다른 손실을 지정할 수 있음

In [59]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[keras.losses.MeanSquaredError(), keras.losses.CategoricalCrossentropy()],
)

모델에 단일 손실 함수만 전달하는 경우, 모든 출력에 동일한 손실 함수가 적용 (여기서는 적합하지 않음).

메트릭의 경우도 마찬가지 ,,

In [60]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[keras.losses.MeanSquaredError(), keras.losses.CategoricalCrossentropy()],
    metrics=[
        [
            keras.metrics.MeanAbsolutePercentageError(),
            keras.metrics.MeanAbsoluteError(),
        ],
        [keras.metrics.CategoricalAccuracy()],
    ],
)

출력 레이어에 이름을 지정했으므로 사전을 통해 출력별 손실과 메트릭을 지정 가능

In [61]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={
        "score_output": keras.losses.MeanSquaredError(),
        "class_output": keras.losses.CategoricalCrossentropy(),
    },
    metrics={
        "score_output": [
            keras.metrics.MeanAbsolutePercentageError(),
            keras.metrics.MeanAbsoluteError(),
        ],
        "class_output": [keras.metrics.CategoricalAccuracy()],
    },
)

출력이 두 개 이상인 경우 명시적 이름과 사전을 사용하는 것이 좋음

`loss_weights` 인수를 사용하여 출력별 손실에 서로 다른 가중치를 부여할 수 있음 <br>
(예를 들어, 클래스 손실에 2x의 중요도를 부여하여 이 예에서 "score" 손실에 우선권을 줄 수 있음)

In [62]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={
        "score_output": keras.losses.MeanSquaredError(),
        "class_output": keras.losses.CategoricalCrossentropy(),
    },
    metrics={
        "score_output": [
            keras.metrics.MeanAbsolutePercentageError(),
            keras.metrics.MeanAbsoluteError(),
        ],
        "class_output": [keras.metrics.CategoricalAccuracy()],
    },
    loss_weights={"score_output": 2.0, "class_output": 1.0},
)

이러한 출력이 예측를 위한 것이고 훈련과는 관계가 없는 경우, 특정 출력에 대한 손실을 계산하지 않도록 선택할 수도 있dma

In [63]:
# List loss version
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[None, keras.losses.CategoricalCrossentropy()],
)

# Or dict loss version
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={"class_output": keras.losses.CategoricalCrossentropy()},
)

`fit()`에서 다중 입력 또는 다중 출력 모델로 데이터를 전달하는 것은 컴파일에서 손실 함수를 지정하는 것과 유사한 방식으로 작동. 즉, <strong>NumPy 배열 목록</strong>(손실 함수를 수신한 출력에 1:1 매핑) 또는 **출력 이름을 NumPy 배열에 매핑한 사전**을 전달할 수 있다!

In [64]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[keras.losses.MeanSquaredError(), keras.losses.CategoricalCrossentropy()],
)

# Generate dummy NumPy data
img_data = np.random.random_sample(size=(100, 32, 32, 3))
ts_data = np.random.random_sample(size=(100, 20, 10))
score_targets = np.random.random_sample(size=(100, 1))
class_targets = np.random.random_sample(size=(100, 5))

# Fit on lists
model.fit([img_data, ts_data], [score_targets, class_targets], batch_size=32, epochs=1)

# Alternatively, fit on dicts
model.fit(
    {"img_input": img_data, "ts_input": ts_data},
    {"score_output": score_targets, "class_output": class_targets},
    batch_size=32,
    epochs=1,
)

4/4 [==============================] - 1s 7ms/step - loss: 17.5895 - score_output_loss: 0.5768 - class_output_loss: 17.0127


다음은 `Dataset`를 사용한 예 -> 사전 튜플을 반환해야 함

In [65]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (
        {"img_input": img_data, "ts_input": ts_data},
        {"score_output": score_targets, "class_output": class_targets},
    )
)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

model.fit(train_dataset, epochs=1)

2/2 [==============================] - 1s 17ms/step - loss: 17.2784 - score_output_loss: 0.3327 - class_output_loss: 16.9457


## 콜백 사용하기

Keras의 콜백은 훈련 중 다른 시점(epoch의 시작, 배치의 끝, epoch의 끝 등)에서 호출되며 다음과 같은 특정 동작을 구현하는 데 사용할 수 있는 객체입니다.

- 훈련 중 서로 다른 시점에서 유효성 검사 수행(내장된 epoch당 유효성 검사에서 더욱 확장)
- 정기적으로 또는 특정 정확도 임계값을 초과할 때 모델 검사점 설정
- 훈련이 평탄해진 것으로 보일 때 모델의 학습률 변경
- 훈련이 평탄해진 것으로 보일 때 최상위 레이어의 미세 조정 수행
- 훈련이 종료되거나 특정 성능 임계값이 초과된 경우 이메일 또는 인스턴트 메시지로 알림 보내기
- 기타

콜백은 `fit()`에 대한 호출에 목록으로 전달할 수 있습니다.

In [66]:
model = get_compiled_model()

callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=2,
        verbose=1,
    )
]
model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=64,
    callbacks=callbacks,
    validation_split=0.2,
)

Epoch 1/20
625/625 [==============================] - 2s 3ms/step - loss: 0.3713 - sparse_categorical_accuracy: 0.8964 - val_loss: 0.2268 - val_sparse_categorical_accuracy: 0.9335
Epoch 2/20
625/625 [==============================] - 2s 3ms/step - loss: 0.1698 - sparse_categorical_accuracy: 0.9496 - val_loss: 0.1686 - val_sparse_categorical_accuracy: 0.9506
Epoch 3/20
625/625 [==============================] - 2s 3ms/step - loss: 0.1245 - sparse_categorical_accuracy: 0.9631 - val_loss: 0.1439 - val_sparse_categorical_accuracy: 0.9579
Epoch 4/20
625/625 [==============================] - 2s 2ms/step - loss: 0.0962 - sparse_categorical_accuracy: 0.9710 - val_loss: 0.1499 - val_sparse_categorical_accuracy: 0.9558
Epoch 5/20
625/625 [==============================] - 2s 3ms/step - loss: 0.0780 - sparse_categorical_accuracy: 0.9758 - val_loss: 0.1354 - val_sparse_categorical_accuracy: 0.9605
Epoch 5: early stopping


### 많은 내장 콜백을 사용할 수 있습니다

Keras에서 이미 사용할 수 있는 내장 콜백은 다음과 같습니다.

- `ModelCheckpoint`: 주기적으로 모델을 저장
- `EarlyStopping`: 훈련이 더 이상 유효성 검사 메트릭을 개선하지 못하는 경우 훈련을 중단
- `TensorBoard`:  시각화할 수 있는 모델 로그를 정기적으로 작성
- `CSVLogger`: 손실 및 메트릭 데이터를 CSV 파일로 스트리밍
- 기타


### 고유한 콜백 작성하기

`self.model`을 통해 연관된 모델에 액세스 가능

In [67]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs):
        self.per_batch_losses = []

    def on_batch_end(self, batch, logs):
        self.per_batch_losses.append(logs.get("loss"))

## 모델 검사점 설정하기

상대적으로 큰 데이터세트에 대한 모델을 훈련시킬 때는 모델의 검사점을 빈번하게 저장하는 것이 중요

이를 수행하는 가장 쉬운 방법은 `ModelCheckpoint` 콜백을 사용하는 것

In [68]:
model = get_compiled_model()

callbacks = [
    keras.callbacks.ModelCheckpoint(
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        # The saved model name will include the current epoch.
        filepath="mymodel_{epoch}",
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="val_loss",
        verbose=1,
    )
]
model.fit(
    x_train, y_train, epochs=2, batch_size=64, callbacks=callbacks, validation_split=0.2
)

Epoch 1/2
601/625 [===========================>..] - ETA: 0s - loss: 0.3956 - sparse_categorical_accuracy: 0.8907
Epoch 1: val_loss improved from inf to 0.27124, saving model to mymodel_1
INFO:tensorflow:Assets written to: mymodel_1\assets


INFO:tensorflow:Assets written to: mymodel_1\assets


625/625 [==============================] - 3s 4ms/step - loss: 0.3899 - sparse_categorical_accuracy: 0.8921 - val_loss: 0.2712 - val_sparse_categorical_accuracy: 0.9176
Epoch 2/2
616/625 [============================>.] - ETA: 0s - loss: 0.1852 - sparse_categorical_accuracy: 0.9455
Epoch 2: val_loss improved from 0.27124 to 0.18012, saving model to mymodel_2
INFO:tensorflow:Assets written to: mymodel_2\assets


INFO:tensorflow:Assets written to: mymodel_2\assets


625/625 [==============================] - 2s 3ms/step - loss: 0.1855 - sparse_categorical_accuracy: 0.9454 - val_loss: 0.1801 - val_sparse_categorical_accuracy: 0.9484


`ModelCheckpoint` 콜백을 사용하여 내결함성을 구현할 수 있습니다. 즉, 훈련이 무작위로 중단되는 경우 모델의 마지막 저장된 상태에서 훈련을 다시 시작할 수 있습니다. 다음은 기본적인 예입니다.

In [69]:
import os

# Prepare a directory to store all the checkpoints.
checkpoint_dir = "./ckpt"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)


def make_or_restore_model():
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    return get_compiled_model()


model = make_or_restore_model()
callbacks = [
    # This callback saves a SavedModel every 100 batches.
    # We include the training loss in the saved model name.
    keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_dir + "/ckpt-loss={loss:.2f}", save_freq=100
    )
]
model.fit(x_train, y_train, epochs=1, callbacks=callbacks)

Creating a new model
  82/1563 [>.............................] - ETA: 2s - loss: 1.1295 - sparse_categorical_accuracy: 0.7092 INFO:tensorflow:Assets written to: ./ckpt\ckpt-loss=1.02\assets


INFO:tensorflow:Assets written to: ./ckpt\ckpt-loss=1.02\assets


 186/1563 [==>...........................] - ETA: 6s - loss: 0.7449 - sparse_categorical_accuracy: 0.8009INFO:tensorflow:Assets written to: ./ckpt\ckpt-loss=0.72\assets


INFO:tensorflow:Assets written to: ./ckpt\ckpt-loss=0.72\assets


 295/1563 [====>.........................] - ETA: 6s - loss: 0.6025 - sparse_categorical_accuracy: 0.8362INFO:tensorflow:Assets written to: ./ckpt\ckpt-loss=0.60\assets


...

1378/1563 [=========================>....] - ETA: 1s - loss: 0.3266 - sparse_categorical_accuracy: 0.9048INFO:tensorflow:Assets written to: ./ckpt\ckpt-loss=0.32\assets


INFO:tensorflow:Assets written to: ./ckpt\ckpt-loss=0.32\assets


1483/1563 [===========================>..] - ETA: 0s - loss: 0.3176 - sparse_categorical_accuracy: 0.9077INFO:tensorflow:Assets written to: ./ckpt\ckpt-loss=0.32\assets


INFO:tensorflow:Assets written to: ./ckpt\ckpt-loss=0.32\assets


1563/1563 [==============================] - 11s 7ms/step - loss: 0.3118 - sparse_categorical_accuracy: 0.9093


모델 저장 및 복원을 위한 자체 콜백을 작성할 수도 있습니다.

직렬화 및 저장에 대한 자세한 설명은 [모델 저장 및 직렬화 가이드](https://www.tensorflow.org/guide/keras/save_and_serialize/)를 참조하세요.

## 학습률 일정 사용하기

딥 러닝 모델을 훈련할 때 일반적인 패턴은 훈련이 진행됨에 따라 점차적으로 학습을 줄이는 것입니다. 이것을 일반적으로 "학습률 감소"라고 합니다.

학습 감소 일정은 정적(현재 epoch 또는 현재 배치 인덱스의 함수로 미리 고정됨) 또는 동적(모델의 현재 동작, 특히 유효성 검사 손실에 대응)일 수 있습니다.

### 최적화 프로그램으로 일정 전달하기

최적화 프로그램에서 schedule 객체를 `learning_rate` 인수로 전달하여 정적 학습률 감소 일정을 쉽게 사용할 수 있습니다.

In [70]:
initial_learning_rate = 0.1
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)

`ExponentialDecay` , `PiecewiseConstantDecay` , `PolynomialDecay` 및 `InverseTimeDecay`와 같은 몇 가지 내장된 일정을 사용할 수 있습니다.

### 콜백을 사용하여 동적 학습률 일정 구현하기

최적화 프로그램이 유효성 검사 메트릭에 액세스할 수 없으므로 이러한 일정 객체로는 동적 학습률 일정(예: 유효성 검사 손실이 더 이상 개선되지 않을 때 학습률 감소)을 달성할 수 없습니다.

그러나 콜백은 유효성 검사 메트릭을 포함해 모든 메트릭에 액세스할 수 있습니다! 따라서 최적화 프로그램에서 현재 학습률을 수정하는 콜백을 사용하여 이 패턴을 달성할 수 있습니다. 실제로 이 부분이`ReduceLROnPlateau` 콜백으로 내장되어 있습니다.

## 훈련 중 손실 및 메트릭 시각화하기

훈련 중에 모델을 주시하는 가장 좋은 방법은 로컬에서 실행할 수 있는 브라우저 기반 애플리케이션인 [TensorBoard](https://www.tensorflow.org/tensorboard)를 사용하는 것입니다. 이 보드에는 다음과 같은 정보가 제공됩니다.

- 훈련 및 평가를 위한 손실 및 메트릭을 실시간으로 플롯
- (옵션) 레이어 활성화 히스토그램 시각화
- (옵션) `Embedding` 레이어에서 학습한 포함된 공간의 3D 시각화

pip와 함께 TensorFlow를 설치한 경우, 명령줄에서 TensorBoard를 시작할 수 있습니다.

```
tensorboard --logdir=/full_path_to_your_logs
```

### TensorBoard 콜백 사용하기

TensorBoard를 Keras 모델 및 `fit()` 메서드와 함께 사용하는 가장 쉬운 방법은 `TensorBoard` 콜백입니다.

가장 간단한 경우로, 콜백에서 로그를 작성할 위치만 지정하면 바로 로그를 작성할 수 있습니다.

In [71]:
keras.callbacks.TensorBoard(
    log_dir="/full_path_to_your_logs",
    histogram_freq=0,  # How often to log histogram visualizations
    embeddings_freq=0,  # How often to log embedding visualizations
    update_freq="epoch",
)  # How often to write logs (default: once per epoch)